# Práctica 1
# Clasificación Lineal y Evaluación del Desempeño
## [Introducción a los Sistemas Inteligentes](https://fagonzalezo.github.io/iis-2018-2/)
### Grupos de máximo 2 personas
### Entrega: Miércoles 26 de Septiembre 2018

En esta práctica vamos a construir un clasificador lineal para clasificar tipos de vinos. Para esto vamos a  descargar la base de datos llamada *Wine* disponible en [aquí](https://drive.google.com/open?id=0B8POkZpAJ5qQRlFzVUZnMlZVakk). Si requiere más información sobre la base datos, la puede encontrar en el siguiente link  [aquí](https://archive.ics.uci.edu/ml/datasets/wine).

El conjunto de datos contiene los ejemplos de 178 vinos. Cada uno identificado por 13 propiedades químicas:
* Alcohol
* Malic_acid
* Ash
* Alcalinity
* Magnesium
* Phenols
* Flavanoids
* Nonflavanoid
* Proanthocyanins
* Color
* Hue
* OD280/OD315
* Proline

A cada vino le fue asignada una clase, que corresponde al tipo de vino $[1, 2, 3]$

1. Cargue el conjunto de datos como un DataFrame de Pandas
2. Cargue la clase en un arreglo aparte. Eliminela del DataFrame de Pandas
3. Genere un histograma para cada propiedad del conjunto de datos
4. Use `scatter_matrix` y visualize todas las posibles combinaciones de características
    * ¿Encuentra algún par de características en el que se observan las clases bien separadas?

**Clasificación usando solo dos características y el esquema Multinomial:**
5. Entrene un modelo LogisticRegression para categorizar el tipo de vino usando `Proline` contra `Flavonoids`.
6. Entrene otro modelo LogisticRegression para categorizar el tipo de vino usando `Alcalinity` contra `Malic Acid`.
7. Imprima la región de decisión de cada clasificador.
8. Genere la matriz de confusión de cada clasificador.
9. Reporte el accuracy, el error de clasificación, la precisión macro, el recall macro y el F1 score macro.
    * ¿Cual modelo se desempeña mejor? ¿Por qué?

**Clasificación usando todas las características:**
10. Entrene un modelo LogisticRegression usando el esquema "*One vs All*".
11. Entrene un modelo LogisticRegression usando el esquema multinomial.
12  Genere la matriz de confusión de cada clasificador.
13  Reporte el accuracy, el error de clasificación, la precisión por clase, el recall por clase y el F1 score por clase.
  * ¿Cual modelo se desempeña mejor? ¿Por qué? ¿Cual clase es más sencilla para el clasificador?

#### Importamos las librerias principales a usar en el notebook ####

In [ ]:
#Instalación del paquete tabulate para imprimir la matriz de confusión
!pip install tabulate

In [ ]:
%matplotlib inline
import numpy as np
import pylab as pl
import seaborn as sns; sns.set()
from sklearn import preprocessing
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from tabulate import tabulate

#### Definimos las funciones a utilizar en el notebook####

In [ ]:
def plot_data(X, y):   #Función para graficar datos (X,y)
    y_unique = np.unique(y)
    colors = pl.cm.rainbow(np.linspace(0.0, 1.0, y_unique.size))
    for this_y, color in zip(y_unique, colors):
        this_X = X[y == this_y]
        pl.scatter(this_X[:, 0], this_X[:, 1],  c=color,
                    alpha=0.5, edgecolor='k',
                    label="Class %s" % this_y)
    pl.legend(loc="best")
    pl.title("Data")

def plot_decision_region2(X, pred_fun):      #Función para visualizar la superficie de decisión de nuestro algoritmo.
    min_x = np.min(X[:, 0])
    max_x = np.max(X[:, 0])
    min_y = np.min(X[:, 1])
    max_y = np.max(X[:, 1])
    min_x = min_x - (max_x - min_x) * 0.05
    max_x = max_x + (max_x - min_x) * 0.05
    min_y = min_y - (max_y - min_y) * 0.05
    max_y = max_y + (max_y - min_y) * 0.05
    x_vals = np.linspace(min_x, max_x, 100)
    y_vals = np.linspace(min_y, max_y, 100)
    XX, YY = np.meshgrid(x_vals, y_vals)
    grid_r, grid_c = XX.shape
    ZZ = np.zeros((grid_r, grid_c))
    for i in range(grid_r):
        for j in range(grid_c):
            ZZ[i, j] = pred_fun(XX[i, j], YY[i, j])
    pl.contourf(XX, YY, ZZ, 100, cmap = pl.cm.coolwarm, vmin= 1, vmax=3)
    pl.colorbar()
    pl.xlabel("x")
    pl.ylabel("y")
    
def gen_pred_fun(clf):
    def pred_fun(x1, x2):
        x = np.array([[x1, x2]])
        return clf.predict(x)[0]
    return pred_fun

def list_cm(cm, classes):     #función para generar de una forma más visual la matriz de confusión
    if len(cm)==2:
      cm.astype(int)
      row_0 =['','Valor','Verdadero']
      row_1 =['-',classes[0],classes[1]]
      row_2 =[classes[0],cm[0,0],cm[1,0]]
      row_3 =[classes[1],cm[0,1],cm[1,1]]
      table = zip(row_0,row_1, row_2, row_3)
      headers = ['', '', 'Valor', 'Predicho']  
      return print(tabulate(table, headers=headers, floatfmt=".0f"))
    else:
      cm.astype(int)
      row_0 =['','Valor','Verdadero','']
      row_1 =['-',np.int(classes[0]),classes[1],classes[2]]
      row_2 =[classes[0],cm[0,0],cm[1,0],cm[2,0]]
      row_3 =[classes[1],cm[0,1],cm[1,1],cm[2,1]]
      row_4 =[classes[2],cm[0,2],cm[1,2],cm[2,2]]
      table = zip(row_0,row_1, row_2, row_3, row_4)
      headers = ['', '', 'Valor', 'Predicho', '']  
      return print(tabulate(table, headers=headers, floatfmt=".0f")) 

### 1. Cargamos el conjunto de datos como un DataFrame de Pandas ###

In [ ]:
!pip install -U -q PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Autenticación y creación del cliente PyDrive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
file_list = drive.ListFile({'q': "'1X_IvhuG5nli8bUly-6PPJovQRGCSq0lH' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

In [ ]:
train_downloaded = drive.CreateFile({'id': '11pvH069hKkdzFC0HYQQ6v0fwAjzdYEd5'})
train_downloaded.GetContentFile('wine.data.txt')

In [ ]:
import pandas as pd
wine_df = pd.read_csv('wine.data.txt')
wine_df.head()

### 2. Cargamos la clase en un arreglo aparte y la eliminamos del DataFrame de Pandas###

### 3. Generamos un histograma para cada propiedad del conjunto de datos###

### 4. Usamos `scatter_matrix` y visualizamos todas las posibles combinaciones de características###
    * ¿Se encuentra algún par de características en el que se observan las clases bien separadas?

# Clasificación usando solo dos características y el esquema Multinomial:

### 5. Entrenamos un modelo LogisticRegression para categorizar el tipo de vino usando `Proline` contra `Flavonoids`.###

### 6. Entrenamos otro modelo LogisticRegression para categorizar el tipo de vino usando `Alcalinity` contra `Malic Acid`.###

### 7. Imprimimos la región de decisión de cada clasificador.###

### 8. Generamos la matriz de confusión de cada clasificador.###

### 9. Reportamos el accuracy, el error de clasificación, la precisión macro, el recall macro y el F1 score macro.###
    * ¿Cual modelo se desempeña mejor? ¿Por qué?

# Clasificación usando todas las características

### 10. Entrenamos un modelo LogisticRegression usando el esquema "*One vs All*".###

### 11. Entrenamos un modelo LogisticRegression usando el esquema multinomial.###

### 12  Generamos la matriz de confusión de cada clasificador.###

### 13  Reportamos el accuracy, el error de clasificación, la precisión por clase, el recall por clase y el F1 score por clase.###
  * ¿Cual modelo se desempeña mejor? ¿Por qué? ¿Cual clase es más sencilla para el clasificador?

**Instrucciones de envío:**

Este notebook debe enviarse a través del siguiente [File Request](https://www.dropbox.com/request/M4GoStgQHPycQ5NwjQba)
antes de la medianoche de la fecha límite. El archivo debe nombrarse como 
`iis-practica1-unalusername1-unalusername2.ipynb`, donde `unalusername` es el nombre de usuario asignado por la universidad (incluya los nombres de usuario de todos los miembros del grupo).